## Import Library 

In [1]:
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
import statistics

Using TensorFlow backend.


## Data Cleaning

In [2]:
df = pd.read_csv('https://cocl.us/concrete_data')
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
df.shape

(1030, 9)

In [4]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
df.columns

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [7]:
df_columns = df.columns

predictors = df[df_columns[df_columns !='Strength']]
target = df['Strength']

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [16]:
n_cols = predictors_norm.shape[1]

## Build a Neural Network

In [17]:
def regression_model():
    
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
   
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [18]:
from sklearn.model_selection import train_test_split

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(predictors_norm, target, test_size=0.30, random_state=1)

print("number of test samples :", X_test_b.shape[0])
print("number of training samples:",X_train_b.shape[0])

number of test samples : 309
number of training samples: 721


## Train and Test the Network

In [19]:
scores_c = []

# looping 50 times over model
for i in range(50):

    # build the model
    model = regression_model()

    # fit the model
    model.fit(X_train_b, y_train_b, validation_data=(X_test_b, y_test_b), epochs=100, verbose=2)

    # evaluate the model
    score_c = model.evaluate(X_test_b, y_test_b, verbose=0)
    scores_c.append(score_c)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 3s - loss: 1507.6326 - val_loss: 1484.5177
Epoch 2/100
 - 1s - loss: 1385.9977 - val_loss: 1327.0877
Epoch 3/100
 - 1s - loss: 1192.4905 - val_loss: 1076.1658
Epoch 4/100
 - 0s - loss: 906.5135 - val_loss: 743.5400
Epoch 5/100
 - 0s - loss: 575.3294 - val_loss: 439.3304
Epoch 6/100
 - 1s - loss: 332.3930 - val_loss: 283.2954
Epoch 7/100
 - 0s - loss: 231.0573 - val_loss: 253.4820
Epoch 8/100
 - 1s - loss: 207.8453 - val_loss: 241.1980
Epoch 9/100
 - 0s - loss: 194.5987 - val_loss: 227.8344
Epoch 10/100
 - 1s - loss: 184.6559 - val_loss: 219.2047
Epoch 11/100
 - 1s - loss: 176.6726 - val_loss: 214.8772
Epoch 12/100
 - 1s - loss: 170.9904 - val_loss: 206.9944
Epoch 13/100
 - 0s - loss: 165.2295 - val_loss: 202.3845
Epoch 14/100
 - 1s - loss: 160.7768 - val_loss: 196.8096
Epoch 15/100
 - 1s - loss: 156.9701 - v

In [20]:
scores_c

[51.17224819837651,
 49.760003605321,
 50.289497363142985,
 47.48703870804179,
 49.30444835922093,
 51.68938426292444,
 45.588268576316466,
 47.817378750896765,
 55.662495782846,
 45.36438741344464,
 50.91206557388059,
 50.153598069373075,
 47.1390863930909,
 48.12069663878012,
 47.864487212838476,
 46.67516220734729,
 47.856072669661934,
 51.950320851841404,
 56.16781903856395,
 48.6119551427156,
 48.2494976574549,
 48.248536353743965,
 50.71862560877136,
 47.3601671607749,
 55.851548944862145,
 46.96476876465634,
 45.74871763210852,
 48.62673897110528,
 47.53219168703147,
 50.988229424436504,
 55.101333099661524,
 51.47064584281452,
 60.63156350145062,
 51.13216449527679,
 48.20364736661942,
 49.79040934738604,
 50.442474735593336,
 48.967147074085226,
 50.56612306119555,
 51.425333165042225,
 44.22060885629993,
 45.9938226965253,
 43.29591200498316,
 43.29989461065496,
 48.537037809304046,
 50.20366315625632,
 45.516968489464816,
 45.89741363031579,
 51.51155496492355,
 49.183175849

In [21]:
print("The mean of normalized data for 100 epochs is", round(statistics.mean(scores_c),1), "and the standard deviation is" , round(statistics.stdev(scores_c),1))

The mean of normalized data for 100 epochs is 49.3 and the standard deviation is 3.4
